In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_train.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_train_comfort_128.joblib")

In [3]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/comfort_train_processed.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["pos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 1 1 ... 1 0 0]
[0 1 1 ... 1 0 0]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[0]


In [5]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/comfort_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_comfort_128.joblib")

In [7]:
# define example
data = dataset_test["pos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 0 0 ... 1 1 0]
[0 0 0 ... 1 1 0]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [17]:
import tensorflow.keras as keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
flatten = Flatten() (input_emb)
output = Dense(2,activation="softmax") (flatten)

m = keras.Model(inputs=[input_emb], outputs=output)

opt = keras.optimizers.RMSprop(lr=0.0001)

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

class myCallback(keras.callbacks.Callback):
    bestF1=0
    def __init__(self):
        super().__init__()
        self._get_pred = None
        self.preds = []
    def on_epoch_end(self,epoch, logs=None):
        predictions = m.predict([emb_matrix_test])
        predictions_vec = np.zeros((1171,4))
        i = 0
        confidences= []
        for item in predictions:
          index = np.argmax(item);
          confidences.append(item[index])
          predictions_vec[i][index] = 1
          i = i+1
        labels_to_eval = []
        for vec in predictions_vec:
          inverted = label_encoder.inverse_transform([argmax(vec)])[0]
          labels_to_eval.append(inverted)
        
        precision,recall,fscore,support=score(dataset_test["pos"],labels_to_eval,average='macro')
        print("Test f1-score:"+str(fscore))
        if fscore > self.bestF1:
          m.save("bestModel.h5")
          print("BEST MODEL!!!!!")
          self.bestF1 = fscore

callbacks = [myCallback()]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
m.summary()

In [18]:
m.fit([emb_matrix_train],onehot_encoded,batch_size=128, epochs=10,validation_split=0.15, callbacks=[callbacks])
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_neg_sentipolc_0.72803.h5")

Epoch 1/10
17/18 [===========================>..] - ETA: 0s - loss: 0.5371 - accuracy: 0.7688Test f1-score:0.8331226687485285
BEST MODEL!!!!!
18/18 [==============================] - 2s 82ms/step - loss: 0.5276 - accuracy: 0.7752 - val_loss: 0.3205 - val_accuracy: 0.8753
Epoch 2/10
17/18 [===========================>..] - ETA: 0s - loss: 0.2617 - accuracy: 0.9173Test f1-score:0.8929680242140909
BEST MODEL!!!!!
18/18 [==============================] - 1s 69ms/step - loss: 0.2634 - accuracy: 0.9158 - val_loss: 0.2676 - val_accuracy: 0.9027
Epoch 3/10
18/18 [==============================] - 1s 65ms/step - loss: 0.2130 - accuracy: 0.9365 - val_loss: 0.2783 - val_accuracy: 0.8928
Epoch 4/10
16/18 [=========================>....] - ETA: 0s - loss: 0.1767 - accuracy: 0.9561Test f1-score:0.9024178283591862
BEST MODEL!!!!!
18/18 [==============================] - 1s 68ms/step - loss: 0.1753 - accuracy: 0.9572 - val_loss: 0.2764 - val_accuracy: 0.9002
Epoch 5/10
18/18 [=========================

In [19]:
m.load_weights("bestModel.h5")
predictions = m.predict([emb_matrix_test])

In [20]:
import numpy as np
predictions_vec = np.zeros((1171,4))
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [21]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["pos"],labels_to_eval,digits=5))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0    0.91791   0.92900   0.92342       662
           1    0.90619   0.89194   0.89901       509

    accuracy                        0.91289      1171
   macro avg    0.91205   0.91047   0.91122      1171
weighted avg    0.91281   0.91289   0.91281      1171



In [23]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_bert_pos.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_creagritrend_neg_full_0.72649_noWMAL.csv");